In [1]:
# Dependencies
import numpy as np
from miditoolkit.midi import parser as mid_parser  
from miditoolkit.midi import containers as ct

In [2]:
# Special tokens
PAD=np.array([2,24,86,64,96,4])
EOS=np.array([4,26,88,66,98,6])
ABS=np.array([5,27,89,67,99,7])

In [3]:
# Token sequence into midi
def token2mid(page,out_path):
    out = mid_parser.MidiFile()
    out.ticks_per_beat = 480
    out.instruments = [ct.Instrument(program=0,is_drum=False,name='reduction')]
    current_beat=-1
    for idx,token in enumerate(page):
        if (token==EOS).all():
            break
        assert((token!=PAD).all())
        if token[0]==0 or (token==ABS).all():
            current_beat+=1
        if (token!=ABS).all():
            n=token
            out.instruments[0].notes.append(ct.Note(start=int(current_beat*4*out.ticks_per_beat+n[1]*out.ticks_per_beat/4),
                                                        end=int(current_beat*4*out.ticks_per_beat+n[1]*out.ticks_per_beat/4+(n[3]+1)*(out.ticks_per_beat/8)),
                                                        pitch=n[2]+22,
                                                        velocity=90))
    out.dump(out_path)

In [4]:
custom_token = np.load('tokens.npy')
first_song = custom_token[0]
first_song

array([[ 0, 16, 29, 27,  0,  3],
       [ 1, 16, 36, 32,  0,  3],
       [ 1, 16, 41,  9,  0,  3],
       ...,
       [ 1, 16, 29, 25,  0,  3],
       [ 1, 16, 41,  2,  0,  3],
       [ 1, 16, 43, 12,  0,  3]])

In [5]:
token2mid(first_song,'out.mid')